## Baseline 2 for clustering similar test steps in natural language 

Assign test steps to the same cluster if the distance (WMD) between their embeddings is zero, i.e., they have the same embeddings
* Text embedding technique: Word2Vec
* Text similarity: Word Mover's Distance (WMD)

In [1]:
# Import necessary libraries
import os
import gc
import pandas as pd
import numpy as np
import math
import statistics as st
import re
import string
import time
import matplotlib.pyplot as plt

# For word frequency
from collections import defaultdict

# ML libraries
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch
import gensim
from gensim.models import Word2Vec, Phrases, KeyedVectors

# NLP libraries
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer, word_tokenize, TweetTokenizer
from nltk.stem import WordNetLemmatizer 

# To be used with hierarchical clustering
from joblib import Memory

# To save models
import pickle

In [ ]:
# Download wordnet data through nltk
nltk.download('wordnet')

### Data preprocessing functions

In [6]:
# Function to compute number of unique words in df
def get_number_unique_words(df):
    words_list = list()
    test_steps = list(df["Steps"])
    for step in test_steps:
        for word in step:
            words_list.append(word)
    number_unique_words = len(set(words_list))
    return number_unique_words

In [7]:
# Function to compute number of unique words in df ('test case name' field)
def get_number_unique_words_name(df):
    words_list = list()
    test_names = list(df["Case_Name"])
    for name in test_names:
        for word in name:
            words_list.append(word)
    number_unique_words = len(set(words_list))
    return number_unique_words

In [8]:
# Function to get list of words that occur less than a certain number of times
def get_word_frequency(df):
    words_list = list()
    test_steps = list(df["Steps"])
    for step in test_steps:
        for word in step:
            words_list.append(word)
    unique_words_list = set(words_list)
    word_occurrence_dict = {}
    for each_word in unique_words_list:
        word_occurrence_dict[each_word] = 0

    for step in test_steps:
        for word in step:
            word_occurrence_dict[word] += 1
            
    ten_times_occurrence_words = list()
    # get list of words that occur only once
    for word, occurrence in word_occurrence_dict.items():
        if occurrence < 2:
            ten_times_occurrence_words.append(word)

    return ten_times_occurrence_words

In [9]:
# Function to get list of words that occur less than a certain number of times ('test case name' field)
def get_word_frequency_name(df):
    words_list = list()
    test_names = list(df["Case_Name"])
    for name in test_names:
        for word in name:
            words_list.append(word)
    unique_words_list = set(words_list)
    word_occurrence_dict = {}
    for each_word in unique_words_list:
        word_occurrence_dict[each_word] = 0

    for name in test_names:
        for word in name:
            word_occurrence_dict[word] += 1
            
    ten_times_occurrence_words = list()
    # get list of words that occur only once
    for word, occurrence in word_occurrence_dict.items():
        if occurrence < 2:
            ten_times_occurrence_words.append(word)

    return ten_times_occurrence_words

In [10]:
# Function to remove problematic/mispelled words from vocabulary
def remove_problematic_words(df):
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps: ", number_unique_words)
    
    # load file with problematic words that exist in the test data
    problematic_words = open('word2vec_vocab_problematic.txt', 'r')
    problematic_words_list = list()
    for word in problematic_words:
        problematic_words_list.append(word.lstrip().rstrip())
    
    for index, row in df.iterrows():
        step = row["Steps"]
        df.loc[index]["Steps"] = [elem for elem in step if not elem in problematic_words_list]
        
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps after removing problematic words: ", number_unique_words)

In [11]:
# function to fix problematic/mispelled words from vocabulary
def fix_problematic_words(df):
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps: ", number_unique_words)
    
    # load file with problematic words that exist in the test data
    problematic_words = open('word2vec_vocab_to_fix.txt', 'r')
    problematic_words_dict = {}
    for line in problematic_words:
        full_line = line.split(':')
        try:
            problematic_words_dict[full_line[0]] = [x.replace('\n', '') for x in full_line[1].split(',')]
        except:
            problematic_words_dict[full_line[0]] = full_line[1].replace('\n', '')
    
    for index, row in df.iterrows():
        step = row["Steps"]
        modified_step = list()
        for word in step:
            if word in problematic_words_dict:
                modified_step.extend(problematic_words_dict[word])
            else:
                modified_step.append(word)
        df.loc[index]["Steps"] = modified_step 
        
    number_unique_words = get_number_unique_words(df)
    print("Number of unique words across all test steps after fixing problematic words: ", number_unique_words)

In [19]:
def preprocess_clean_data(df):
    print("Cleaning test case name field...")
    print("Dataset size before preprocessing: " , df.shape)
    
    # preprocessing and clean test name
    
    # replace url and similar structures (e.g, paths) with the keyword 'URL'
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub(r'http\S+', 'URL', x))
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub('\/[\w-]*', '', x))
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub(r'\{[^)]*\}', '', x))
    
    # lowercase the step descriptions
    df["Case_Name"] = df["Case_Name"].apply(lambda x: x.lower())
    
    # remove digits and words with digits
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub('\w*\d\w*','', x))
    
    # remove punctuations
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

    # remove extra spaces
    df["Case_Name"] = df["Case_Name"].apply(lambda x: re.sub(' +',' ',x))

    # tokenization
    df["Case_Name"] = df["Case_Name"].apply(lambda x: TweetTokenizer().tokenize(x))
    number_unique_words = get_number_unique_words_name(df)
    print("Number of unique words across all test names: ", number_unique_words)
    
    # stopword removal
    stop_words = set(stopwords.words('english'))
    df["Case_Name"] = df["Case_Name"].apply(lambda x: [w for w in x if not w in stop_words])
    number_unique_words = get_number_unique_words_name(df)
    print("Number of unique words in test names after stopword removal: ", number_unique_words)
    
    # lemmatization
    lemmatizer = WordNetLemmatizer() 
    df["Case_Name"] = df["Case_Name"].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
    
    # remove words that occur a certain number of times
    ten_times_occurrence_words = get_word_frequency_name(df)
    print("Number of words that occurred only once in test case names: ", len(ten_times_occurrence_words))
    
    # list of words to be removed
    for index, row in df.iterrows():
        current_test_name = row["Case_Name"]
        list_words_to_remove = list()
        for word in current_test_name:
            if word in ten_times_occurrence_words:
                list_words_to_remove.append(word)
        
        df.loc[index]["Case_Name"] = [elem for elem in current_test_name if not elem in list_words_to_remove]

#     # remove single letters present in the data
#     df["Name"] = df["Name"].apply(lambda x: [w for w in x if len(w.strip()) > 1])

    # remove instances with empty names
    df = df.loc[df["Case_Name"] != '']
    
    number_unique_words = get_number_unique_words_name(df)
    print("Number of unique words in test names in the end: ", number_unique_words)
    print("Dataset size after preprocessing: " , df.shape)

### Read and preprocess files with test cases and build dataframe

In [20]:
# Get data directory and list of xlsx files
current_dir = os.getcwd() 
parent_dir = os.path.dirname(current_dir) + "\\filtered_data\\"
xlsxfiles = [os.path.join(root, name)
             for root, dirs, files in os.walk(parent_dir)
             for name in files
             if name.endswith((".xlsx"))]

In [21]:
# Declare pandas df to be populated
column_names = ["Type", "Key", "Case_Name", "Step_ID", "Steps"]
test_steps_df = pd.DataFrame(columns = column_names)

# Index to add data to the df
index_to_add = 0

print("Reading input data...")   
for test_file in xlsxfiles:
    # load data and iterate through it
    test_data_df = pd.read_excel(test_file)
    for index, row in test_data_df.iterrows():
        current_type = row["Type"]
        current_key = row["Key"]
        current_name = row["Case_Name"]
        current_step_id = row["Step_ID"]
        current_steps = row["Steps"]
        test_steps_df.loc[index_to_add] = [current_type, current_key, current_name, current_step_id, current_steps]
        index_to_add += 1

print("Done!")
print("Shape of data => ", test_steps_df.shape)

Reading input data...
Done!
Shape of data =>  (15668, 5)


In [22]:
# Call preprocessing function
preprocess_clean_data(test_steps_df)

Cleaning test case name field...
Dataset size before preprocessing:  (15668, 5)
Number of unique words across all test names:  1519
Number of unique words in test names after stopword removal:  1447
Number of words that occurred only once in test case names:  164
Number of unique words in test names in the end:  1138
Dataset size after preprocessing:  (15668, 5)


In [23]:
# Preprocess steps here as for some reason its not processing properly within the function
# Replace urls with the keyword 'URL'
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub(r'http\S+', 'URL', x))

# Remove structures similar to urls (e.g, paths) 
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub('\/[\w-]*', '', x))
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub(r'\{[^)]*\}', '', x))

# Lowercase the step descriptions
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: x.lower())

# Remove digits and words with digits
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub('\w*\d\w*','', x))

# Remove punctuations
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

# Remove extra spaces
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: re.sub(' +',' ',x))

# Tokenization
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: TweetTokenizer().tokenize(x))
number_unique_words = get_number_unique_words(test_steps_df)
print("Number of unique words across all test steps: ", number_unique_words)

# Remove and fix mispelled words
remove_problematic_words(test_steps_df)
fix_problematic_words(test_steps_df)

# Stopword removal
stop_words = set(stopwords.words('english'))
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: [w for w in x if not w in stop_words])
number_unique_words = get_number_unique_words(test_steps_df)
print("Number of unique words in test steps after stopword removal: ", number_unique_words)

# Lemmatization
lemmatizer = WordNetLemmatizer() 
test_steps_df["Steps"] = test_steps_df["Steps"].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

# Remove words that occur only once in all test fields
ten_times_occurrence_words = get_word_frequency(test_steps_df)
print("Number of words that occurred only once in test steps: ", len(ten_times_occurrence_words))

for index, row in test_steps_df.iterrows():
    current_test_step = row["Steps"]
    list_words_to_remove = list()
    for word in current_test_step:
        if word in ten_times_occurrence_words:
            list_words_to_remove.append(word)

    test_steps_df.loc[index]["Steps"] = [elem for elem in current_test_step if not elem in list_words_to_remove]

number_unique_words = get_number_unique_words(test_steps_df)
print("Number of unique words in test steps in the end: ", number_unique_words)
print("Dataset size after preprocessing: " , test_steps_df.shape)

Number of unique words across all test steps:  2627
Number of unique words across all test steps:  2627
Number of unique words across all test steps after removing problematic words:  2617
Number of unique words across all test steps:  2617
Number of unique words across all test steps after fixing problematic words:  2599
Number of unique words in test steps after stopword removal:  2488
Number of words that occurred only once in test steps:  507
Number of unique words in test steps in the end:  1742
Dataset size after preprocessing:  (15668, 5)


In [ ]:
# Print head of preprocessed dataset
test_steps_df.head()

In [30]:
# Get only necessary fields to train word embedding models ('type', 'name', 'steps')
# Note: 'type' is always a str, while 'name' and 'steps' are always lists (even if they have a single element)
test_steps_training_list = list()
for index, row in test_steps_df.iterrows():
    temp_list = list()
    if not pd.isnull(row["Type"]):
        temp_list.append(str(row["Type"]))
    
    if isinstance(row["Case_Name"], list):
        for elem in row["Case_Name"]:
            temp_list.append(elem)
    else:
        if isinstance(row["Case_Name"], str):
            temp_list.append(row["Case_Name"])

    if isinstance(row["Steps"], list):
        for elem in row["Steps"]:
            temp_list.append(elem)
    else:
        if isinstance(row["Steps"], str):
            temp_list.append(row["Steps"])
            
    # Build list of lists of tokens (words)
    test_steps_training_list.append(temp_list)

print("Length of list with training data: " , len(test_steps_training_list))

Length of list with training data:  15668


In [31]:
# Check if all elements of the training data is correct (words should be strings)
for step in test_steps_training_list:
    for word in step:
        if not isinstance(word, str):
            print("Error with data type!")
            break

### Train word embedding model - Word2Vec

In [34]:
# Set parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 2   # Minimum word count (this should hold for all existing words now - one-occurring words were already removed)                       
num_workers = 4       # Number of threads to run in parallel (if necessary)
context = 2         # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [35]:
start = time.time()
my_model = Word2Vec(workers=num_workers,
                   size=num_features,
                   min_count = min_word_count,
                   window = context,
                   sample = downsampling)
# default of sg=0 (CBOW) to match the default of the last version of word2vec.c from the original researchers

my_model.build_vocab(test_steps_training_list)
total_examples = my_model.corpus_count
print("Size of initial vocabulary: ", len(my_model.wv.vocab.keys()))

# Load pre-trained model (set your custom path to where the model is located)
pre_trained_model_path = "E:\\TEMPORARY\\laptop\\Prodigy-collab\\Paper1\\GoogleNews-vectors-negative300.bin\\GoogleNews-vectors-negative300.bin"
pre_trained_model = KeyedVectors.load_word2vec_format(pre_trained_model_path, binary=True)

all_words = list(my_model.wv.vocab.keys())
for each_word in all_words:
    my_model.wv[each_word] = np.zeros(300)

# Update vocabulary with our corpus (must set 'lockf' to 1.0 to update model vocabulary)
my_model.build_vocab([list(pre_trained_model.vocab.keys())], update=True)
my_model.intersect_word2vec_format(pre_trained_model_path, binary=True, lockf=1.0)

# Get mean and sd of initialized word vectors
word_vector_median_list = list()
count_initiliazed = 0
for each_word in all_words:
    # if word vector was initialized
    if any(my_model.wv[each_word] != 0):
        word_vector_median_list.append(np.median(my_model.wv[each_word]))
        count_initiliazed += 1
print("Number of words already initialized : " , count_initiliazed)

# Define mean and sd for the normal distributions of medians        
mu = np.mean(word_vector_median_list)
sigma = np.std(word_vector_median_list)

# Initialize the remaining word vectors (not present in the pre-trained model)
count_manual_initiliazed = 0
for each_word in all_words:
    # if word vector was NOT initialized
    if all(my_model.wv[each_word] == 0):
        # initialize this word vector
        new_word_vector = np.random.normal(mu, sigma, 300)
        my_model.wv[each_word] = new_word_vector
        count_manual_initiliazed += 1
print("Number of words manually initialized : " , count_manual_initiliazed)

my_model.train(test_steps_training_list, total_examples=total_examples, epochs=15)
end = time.time()
print(f'Finished in {round(end-start,2)} second(s)')

Size of initial vocabulary:  2032
Number of words already initialized :  1750
Number of words manually initialized :  282
Finished in 77.77 second(s)


In [ ]:
# Examples of how we can use the trained model for word embedding
print(my_model.predict_output_word(['launch', 'prodigy', 'select', 'device']), '\n')
print(my_model.wv.most_similar('epics'), '\n')
print(my_model.wv.doesnt_match(['wand', 'boot', 'house', 'weapon', 'hat']))

In [34]:
# # save vocab (uncomment to save the model vocabulary)
# out_file= open("word2vec_vocab.txt", "a")
# for word in list(my_model.wv.vocab):
#     out_file.write(word + '\n')
# out_file.close()

In [40]:
# Save the model (uncomment to save)
path_save_model = 'appr_1_my_model.model'
# my_model.save(path_save_model)

# Load the model (uncomment to load)
path_load_model = 'appr_1_my_model.model'
# my_model = Word2Vec.load(path_load_model)

### Compute distance (WMD) between all pairs of test steps

In [41]:
# Build tuples with (step_id, step_text) - used to retrieve the step ID in the end (after the clustering) - and get only test steps for clustering
step_id_text_tuple_list = list()
test_steps_clustering_list = list()
for index, row in test_steps_df.iterrows():
    step_id = row["Step_ID"]
    step_text = row["Steps"]
    step_id_text_tuple_list.append((step_id,step_text))

    temp_list = list()
    if isinstance(row["Steps"], list):
        for elem in row["Steps"]:
            temp_list.append(elem)
    else:
        if isinstance(row["Steps"], str):
            temp_list.append(row["Steps"])
        
    # build list of lists of tokens (words)
    test_steps_clustering_list.append(temp_list)
    
print("Length of list of tuples:" , len(step_id_text_tuple_list))
print("Length of list with test steps: " , len(test_steps_clustering_list))

Length of list of tuples: 15668
Length of list with test steps:  15668


In [42]:
# Remove empty steps
index = 0
steps_to_remove = list()
for step in test_steps_clustering_list:
    if len(step) == 0:
        steps_to_remove.append(index)
    index += 1

step_id_text_tuple_list = [step_id_text_tuple_list[index] for index in range(len(step_id_text_tuple_list)) if not index in steps_to_remove]
test_steps_clustering_list = [test_steps_clustering_list[index] for index in range(len(test_steps_clustering_list)) if not index in steps_to_remove]
print("Length of list of tuples:" , len(step_id_text_tuple_list))
print("Length of list with test steps: " , len(test_steps_clustering_list))

Length of list of tuples: 15644
Length of list with test steps:  15644


In [32]:
# Assign the number of test steps to rows and cols and initialize distance matrix with zeros
rows = cols = len(test_steps_clustering_list)
dist_matrix = np.zeros((rows, cols))

In [ ]:
start = time.perf_counter()
print("Computing distances....")
for row in range(rows):
    for col in range(row, cols):
        computed_dist = my_model.wmdistance(test_steps_clustering_list[row], test_steps_clustering_list[col])
        # Upper bound to avoid having inf values
        if computed_dist > 1000:
			computed_dist = 1000
        dist_matrix[row,col] = dist_matrix[col,row] = computed_dist

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

In [ ]:
# # save distance matrix
path_save_dist_matrix = 'appr_1_dist_matrix.txt'
# np.savetxt(path_save_dist_matrix, dist_matrix)

# load distance matrix
path_load_dist_matrix = 'appr_1_dist_matrix.txt'
dist_matrix = np.loadtxt(path_load_dist_matrix)

In [ ]:
# Print matrix's shape and check if it's correct
dist_matrix.shape

### Use WMD = 0 to cluster test steps

In [ ]:
similar_steps_dict = {}
for i in range(len(test_steps_clustering_list)):
    similar_steps_dict[i] = [i]
    print(i)
    for j in range(i,len(test_steps_clustering_list)):
        if dist_matrix[i,j] == 0:
            temp_list = similar_steps_dict[i]
            if j not in temp_list:
                temp_list.append(j)
                similar_steps_dict[i] = temp_list

In [ ]:
same_steps_list = []
found_flag = [False] * len(test_steps_clustering_list)

for i in range(len(test_steps_clustering_list)-1):
    temp_set = set()
    if not found_flag[i]:
        temp_set.add(i)
        found_flag[i] = True
    else:
        continue
        
    for j in range(i+1, len(test_steps_clustering_list)):
        if found_flag[j]:
            continue
        else:
            if test_steps_clustering_list[i] == test_steps_clustering_list[j]:
                temp_set.add(j)
                found_flag[j] = True
    same_steps_list.append(temp_set)

In [106]:
# Print cluster exmaple with more than 10 test steps
for key in similar_steps_dict:
    if len(similar_steps_dict[key]) > 10:
        print(key)
        break

17


In [ ]:
# Merge clusters that share elements
set_list = list()
index_list_skip = list()
for i in range(len(similar_steps_dict)):
    print(i)
    base_set = set(similar_steps_dict[i])
    if i not in index_list_skip:
        if i != len(similar_steps_dict):
            for j in range(i+1, len(similar_steps_dict)):
                compared_set = set(similar_steps_dict[j])
                if len(base_set.intersection(compared_set)) > 0:
                    base_set = base_set.union(compared_set)
                    index_list_skip.append(j)
        set_list.append(base_set)

In [46]:
print("Final number of clusters : ", len(set_list))

Final number of clusters :  4393


In [81]:
# Example of cluster with more than 10 test steps
for each in set_list:
    if len(each) > 10:
        print(each)
        break

{132, 142, 17, 20, 149, 23, 26, 155, 29, 32, 161, 36, 167, 43, 173, 49, 182, 59, 191, 65, 68, 197, 73, 76, 205, 81, 84, 214, 89, 92, 97, 102, 107}


In [49]:
# Save clustered data
path_save_data = "baseline_2/baseline_2_clustered_data.txt"
out_cluster_file = open(path_save_data, "a")
label_cluster = 0
for each_set in set_list:
    for index in each_set:
        str_to_save = "[" + str(label_cluster) + "]:\t\t" + test_steps_df.loc[index]["Key"] + "\t\t" + str(step_id_text_tuple_list[index][0]) + "\t\t" + str(test_steps_clustering_list[index]) + "\n"
        out_cluster_file.write(str_to_save)
    label_cluster += 1
out_cluster_file.close()

In [50]:
# Save cluster labels
path_save_labels = "baseline_2/baseline_2_cluster_labels.txt"
out_cluster_file = open(path_save_labels, "a")
label_cluster = 0
for each_set in set_list:
    str_to_save = "[" + str(label_cluster) + "]: " + ','.join(str(step_id_text_tuple_list[x][0]) for x in each_set) + "\n"
    out_cluster_file.write(str_to_save)
    label_cluster += 1
out_cluster_file.close()

### Compute F-score

#### Load ground truth of similar test steps (to compute F-score)

In [6]:
# Read excel files with manually clustered samples
manual_sample_dir = 'sample_manual_ground_truth/clusters/'
sample_files = os.listdir(manual_sample_dir)

In [3]:
manual_clusters_dict = {}
for sample in sample_files:
    sample_df = pd.read_excel(manual_sample_dir + sample)
    for index, row in sample_df.iterrows():
        cluster_id = row['cluster_id']
        step_id = row['step_id']
        if step_id in manual_clusters_dict:
            existing_list = manual_clusters_dict[step_id]
            existing_list.append(cluster_id)
            manual_clusters_dict[step_id] = existing_list
        else:
            manual_clusters_dict[step_id] = [cluster_id]

In [4]:
print("Number of test step samples which were manually clustered: ", len(manual_clusters_dict))

Number of test step samples which were manually clustered:  394


In [5]:
test_steps_to_evaluate_list = list(manual_clusters_dict.keys())

In [96]:
appr_clusters_dict = {}
cluster_id = 0
for each_set in set_list:
    for elem in each_set:
        step_id = step_id_text_tuple_list[elem][0]
        appr_clusters_dict[int(step_id)] = cluster_id
    cluster_id += 1

In [97]:
print("Number of test steps which were clustered by the approach: ", len(appr_clusters_dict))

Number of test steps which were clustered by the approach:  15644


In [102]:
# Declare and initialize variables to compute F-score
TP = 0
FP = 0
TN = 0
FN = 0

In [103]:
# Iterate through list of test steps to evaluate
for i in range(len(test_steps_to_evaluate_list)-1):
    for j in range(i+1, len(test_steps_to_evaluate_list)):
        step_id_1 = test_steps_to_evaluate_list[i]
        step_id_2 = test_steps_to_evaluate_list[j]
        
        # true positive case
        if (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
            TP += 1
            
        # false positive case
        elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] == appr_clusters_dict[step_id_2]):
            FP += 1
            
        # false negative case
        elif (manual_clusters_dict[step_id_1] == manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
            FN += 1
            
        # true negative case
        elif (manual_clusters_dict[step_id_1] != manual_clusters_dict[step_id_2]) and (appr_clusters_dict[step_id_1] != appr_clusters_dict[step_id_2]):
            TN += 1

In [104]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print("Precision = ", precision)
print("Recall = ", recall)

Precision =  1.0
Recall =  0.5432098765432098


In [105]:
f_score = (2 * precision * recall) / (precision + recall)
print("F-score = ", f_score)

F-score =  0.704
